In [1]:
# TODO: We'll need to get scrapy installed onto the images
# !pip install --user scrapy
# !pip install --user plotly
# !pip install --user textblob # this should also install nltk

In [2]:
import requests

from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from scrapy import Request

import plotly.plotly as py

from IPython.core.display import HTML

from notebook_code.scraping_support import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ackerman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Introduction

Why on earth would you want to scrape a web page?

In an ideal world, you wouldn't have to. Everyone would realize that pages they put up on the web contained information that was useful beyond the ways they imagined, and while they built their web pages to present that information the way they wanted to, they'd *also* provide a way to get just that raw data in a way that is easy for machines to process. Things are getting better as everyone realizes the interesting things we can do with the raw data and all levels of government are making their data more open and accessible. But we're not quite there yet, and web scraping is often the necessary glue we need to use to stick things together while we wait.

By the end of this notebook, we'll be able to grab the quotations from [Quotes to Scrape](http://quotes-to-scrape.com) and print them out directly, without all of the other information and formatting on the page that we don't want. We'll also be able to count up the number of quotes per author and display that information!

Here's what the page looks like:

![website-to-scrape](images/website-to-scrape.png)


## Ethical Scraping

It's important to think about what you're doing when you're scraping a website. You're collecting information that someone else or many people took a long time to gather. Your scraping code will also access information on their website much more quickly than a human would. This could cause problems for the website if you're not careful. This could result in your IP address being blocked by the website or more serious measures to stop you from hurting their quality of service. You should definitely check for any official notices the website may make discouraging scraping and respect them. Even if you don't see a notice, you should avoid scraping information simply to show exactly the same information on your own site. You should be doing something meaningful to that information and give credit and link to the source of it.

You should also be careful about designing your scrapers. Don't leave scrapers running automatically that you're not sure are "good web citizens". Also, avoid running them too often. Do you really need to update things every few minutes? Or could you do it once a day? How about once a week? Or only when you rerun a cell in a notebook (like we're doing here).

## A Crash Course in HTML

To properly scrape a page, you first need to know a little about HTML (also known as: hypertext markup language). It's what almost every website you'll find out there is built on.

The main thing that makes HTML different than regular text is its use of **tags**. A `<` marks the beginning of a tag and a `>` marks the end of a tag. If you're just using a single tag on its own, it should end with a `/>`. For example, a line break is `<br/>`. Your browser doesn't need to know anything else about it to know it should put an empty line between that tag and whatever comes next.

However, most tags are a little more sophisticated than our rather boring `<br/>`. Most tags do *something* to what  comes after them. For example `<b>` tells you to start **making everything bold. But eventually bold words just get distracting, so you'll want to stop making them all bold. You do that with `</b>`**.

Notice that the only difference between the closing tag and the opening tag is that the closing tag begins with `</` instead of `<`. 

When your browser is figuring out how to display a page, it expects that you provide the closing tag whenever you use an opening tag. You can put whatever you want in between the opening and closing tags as long as it's valid HTML.

For example, `<i>foo</i>` will italicize *foo*. If you want to also make **<i>foo</i>** bold, you could put the valid HTML `<i>foo</i>` inside an opening and closing `b` tags: 

```html
<b><i>foo</i></b>
```

In case you're wondering, yes, you could also get the same result with:

```html
<i><b>foo</b></i>
```

That may seem like a lot of work to do something you could do in a couple of mouse clicks or key presses in your favourite word processor. The beauty of HTML is that all web browsers are expected to do the *same* thing when they run into any of the standard HTML tags. That means that the tags you wrote to make your text bold and italicized will result in **<i>bold, italicized</i>** text no matter what browser someone uses to read it.

### Valid HTML Documents

We've learned what basic valid HTML looks like. What about a whole HTML document? Here's the minimum an HTML document should have in order to be valid HTML5 (the latest version of HTML):

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Your page title goes here.</title>
  </head>
  <body>
    Your main page content goes here.
  </body>
</html>
```

Web browsers can be pretty forgiving, and your page probably won't break if you miss some of these. Don't worry about trying to memorize this either. You can find it in more than enough places on line and it's perfectly okay to cut and paste to start out. Even the pros do that.

We'll just be focusing on what happens within the `<body></body>` tags, so for the following examples, just remember that to make them into a valid HTML document, you'd have to put them inside the `<body></body>` tags above.

## A Simple Scraper

There's a lot more we could say about HTML and its standard tags, and if you're interested, you can read about those tags [here](https://www.w3schools.com/tags/default.asp). Along with CSS (or cascading style sheets), another standard expected to be supported by every browser, you can make text and images look exactly the way you want in any web browser. And with a third standard language, Javascript, you can even create sophisticated user interfaces that people can interact with. Bet your word processor can't do that!

However, this notebook's purpose is to get you ready to scrape other peoples' nice looking web pages! And we now know enough about HTML to start doing that. So let's get into it!

### Your First Mission

Let's say we have the following within our document `<body></body>`:

```html
The answer to the question is: <b>42</b>.
```

Here's what that looks like:

In [3]:
html_text = 'The answer to the question is: <b>42</b>.'
HTML(html_text)

How would you get *just* the answer `42`?

Well, you *could* just look for some part of the text before and after it. For example, you could look for `'is: '` and grab everything until the first `.`. That's after you remove that bold tag and any other HTML tags within the text:

In [4]:
raw_text = remove_tags(html_text)
before_text = 'is: '
answer_start = raw_text.find(before_text) + len(before_text)
answer_end = raw_text.find('.', answer_start)
raw_text[answer_start:answer_end]

'42'

What are some problems with finding the answer this way? Well, first, it's a lot of work! But on top of that, if we change any of the text around the answer, we risk breaking the code we're using to grab it. What if, instead of stripping out the `<b>` tag, we used it as a way to locate the answer? That's where scraping libraries like *scrapy* come in. They can use the structure of the HTML itself to help drill down to the parts of it we're interested in.

Here's how we can find our answer using *scrapy*:

In [5]:
Selector(text=html_text).xpath('//b/text()').extract_first()

'42'

### XPaths

What's going on here? Let's break up that first line above into 3 pieces:

1. `Selector(text=html_text)`
2. `xpath('//b/text()')`
3. `extract_first()`

The first piece is just a way of initializing **scrapy** with the HTML that we want it to dig into. We're creating the `Selector` object with our `html_text` string, and that object will do the hard work of figuring out the structure of that text string based on all the HTML tags, etc. in it. The last piece tells **scrapy** that we want to return the first match to our query. We could have used `extract` instead of `extract_first`, and received an array with a single element (`'42'`) that we would then have to pull out of the array.

As important as the first and third pieces are, the second piece is where we'll be spending most of our time. This is where you tell **scrapy** what to look for and where to start looking for it.

Let's modify our example just a bit:

In [6]:
html_text = '''
Answer to Question 1: <b>42</b><br/>
Answer to Question 2: <b>2</b><br/>
Answer to Question 3: <b>foo</b>
'''
HTML(html_text)

How would you get all 3 answers? Note that we've also changed the text that comes before and after the answers. This would completely break our very first code for extracting the answer. How much would you have to change the **scrapy** version?

The great thing is you don't have to change either the first or second part at all! The answers are all still within those `<b></b>` tags. But now, instead of just as single match, we want *all* of them.

In [7]:
Selector(text=html_text).xpath('//b/text()').extract()

['42', '2', 'foo']

What if we put those answers inside another HTML element? We're going to use a `<div></div>` element that's kind of like an all-purpose container for things. Think of it as a box that you can put stuff in. That box can go into bigger boxes, and those boxes can be moved around without repositioning the things inside them. Here's what our answers look like inside a `div`:

In [8]:
html_text = '''
<div>
    Answer to Question 1: <b>42</b><br/>
    Answer to Question 2: <b>2</b><br/>
    Answer to Question 3: <b>foo</b>
</div>
'''
HTML(html_text)

Nothing *visually* has changed about the text. Right now, our `div` is an invisible box. So, does that change what we need to do to find our answers? Nope!

In [9]:
Selector(text=html_text).xpath('//b/text()').extract()

['42', '2', 'foo']

It seems like there might be just a bit of magic going on with that small bit of text within our `xpath` function call. Let's take a closer look at it. What does `'//b/text()'` mean to **scrapy's** `xpath` function? Just like we did above, let's pull out the pieces:

1. `//` and `/`
2. `b`
3. `text()`

Let's talk about parts 2 and 3 first. Those are pretty straightforward. `b` is just the name of the tag, without needing to put in any of the `<>`. `text()` is a special xpath function that tells it to look for a plain text element.

`//` and `/` both mean to look within the current tag's children. If nothing is on the left, the "current tag" is the root of the document. The difference is that while `/` expects the tag on the right to be a direct child of the tag on the left, `//` will keep checking the children of children, retreiving anything that matches. Neither is better than the other. In fact, you'll likely need to play around a bit with them to find the right combination that is *specific enough* to not return a bunch of results you aren't interested in, but *general enough* that if someone made a small change to the website you're scraping, your scraper wouldn't break.

Without any `//` you'd have to write out the exact path to get the same answer as above. Try it!:

In [10]:
Selector(text=html_text).xpath('/html/body/div/b/text()').extract()

['42', '2', 'foo']

Now try taking out one of those tags (we'll take out the `div` tag:

In [11]:
Selector(text=html_text).xpath('/html/body/b/text()').extract()

[]

Where could you add a single `//` to the above xpath string to get those results back?

In [12]:
Selector(text=html_text).xpath('/html/body//b/text()').extract()

['42', '2', 'foo']

This means your `<b></b>` tags around the answers can be anywhere inside `<html><body></body></html>`, but not outside of it. If the `<div>` tag was replaced with `<p>` (for 'paragraph') or any other tag, your scraper would still work. If the answers were put another level deeper, the scraper would still find them.

## Using HTML attributes

In addition to childern, any HTML tag can have a number of attributes. Here's what they look like:

```html
<some_tag attribute_1='value_1' attribute_2='value_2'>
  ...
</some_tag>
```

One attribute that has special meaning and is used frequently is `class`. `class` is used to tell the browser to use certain CSS style information when drawing the tag and its children. A tag can have many classes, each separated by a space. For example:

```html
<some_tag class='class_1 class_2 class_3'>
  ...
</some_tag>
```

We're not going to get into how CSS works here, though. What's important for our scraper is that these classes are used a lot and often have names that are related to the kind of information they contain. This isn't a rule. They can really be anything you want. But the reason people will often use the type of information as a class name is that they don't want to have to think about the structure of the page and where that tag is just to change some small part of its appearance (its color, for example). And because of this, they can help us make our scrapers more specific and less fragile.

Let's go back to our sample HTML and change it a bit:

In [13]:
html_text = '''
<div>
    Answer to Question 1: <b class='answer'>42</b><br/>
    Answer to Question 2: <b class='answer'>2</b><br/>
    Answer to Question 3: <b class='answer'>foo</b><br/>
    This isn't an answer to anything: <b>Just a little bold text!</b>
</div>
'''
HTML(html_text)

What happens if we run the same scraping code we did above on this?

In [14]:
Selector(text=html_text).xpath('//b/text()').extract()

['42', '2', 'foo', 'Just a little bold text!']

Aha! We still get our answers, but we also get something we don't want, just because it happens to be within a `<b></b>` tag. Let's add knowledge of the 'answer' class to our scraper so it just picks up the right pieces:

In [15]:
xpath_string = '//b[contains(concat(" ", normalize-space(@class), " ")," answer ")]/text()'
Selector(text=html_text).xpath(xpath_string).extract()

['42', '2', 'foo']

That's a little complicated. It's handling the possibility that one or more of the `<b>` tags may have *multiple* classes, and it's making sure to not match class names that just contain 'answer' (such as 'not_an_answer'). This is where scrapy's `css` function and the ability to chain your scraping calls comes in handy.

Thinking about it another way, we could ask to find all the `<b></b>` tags, then just find the tags within that set that have the class `answer`, then find the text underneath:

In [16]:
Selector(text=html_text).xpath('//b').css('.answer').xpath('.//text()').extract()

['42', '2', 'foo']

The `css` function uses [CSS selectors](https://www.w3schools.com/cssref/css_selectors.asp) to navigate tags.

## Exercises

There's a lot more to *xpaths* and *CSS selectors* that can help you dig through a page to get just the right information, but you can do a lot with what you've already learned. There's more than one way to get to the information you need and there are no absolute rules about how to do it.

Here's some more complicated HTML, similar to what you might see on a real webpage:

In [17]:
html_text = '''
<div class='main'>
    <div class='navigation'>
        <ul>
            <li>Menu item 1</li>
            <li>Menu item 2</li>
            <li>Menu item 3</li>
            <li>Menu item 4</li>
        </ul>
    </div>
    <div class='details'>
        <div class='section paragraphs'>
            <h1>A bunch of paragraphs</h1>
            <p>This is the first paragraph</p>
            <p>This is the second paragraph</p>
            <p>This is the third paragraph</p>
        </div>
        <div class='section table'>
            <h1>A table of data</h1>
            <p>This is in a paragraph tag but shouldn't show up in the exercises</p>
            <table class='data'>
                <thead>
                    <tr><td>A</td><td>B</td><td>C</td></tr>
                </thead>
                <tbody>
                    <tr><td>1</td><td>2</td><td>3</td></tr>
                    <tr><td>2</td><td>4</td><td>6</td></tr>
                    <tr><td>3</td><td>6</td><td>9</td></tr>
                    <tr><td>4</td><td>8</td><td>12</td></tr>
                </tbody>
            </table>
        </div>
    </div>
</div>
'''
HTML(html_text)

A,B,C
1,2,3
2,4,6
3,6,9
4,8,12


### Question 1:

Extract just the 3 paragraphs under the **A bunch of paragraphs** section.

In [18]:
Selector(text=html_text).css('.details .section.paragraphs').xpath('.//p/text()').extract()

['This is the first paragraph',
 'This is the second paragraph',
 'This is the third paragraph']

### Question 2:

Extract all of the numbers in column B of the table. Hint: `td[2]` instead of just `td` in your xpath will tell scrapy to always grab the 2nd `td` tag under every `tr` (or row) tag. Using a CSS selector, `td:nth-child(2)` does the same thing.

In [19]:
Selector(text=html_text).css('.details table tbody tr td:nth-child(2)').xpath('.//text()').extract()

['2', '4', '6', '8']

In [20]:
Selector(text=html_text).xpath('//table/tbody/tr/td[2]/text()').extract()

['2', '4', '6', '8']

### Question 3:

Extract just the headings.

In [21]:
Selector(text=html_text).xpath('//h1/text()').extract()

['A bunch of paragraphs', 'A table of data']

### Question 4:

Extract just the menu items.

In [22]:
Selector(text=html_text).css('.navigation').xpath('.//ul/li/text()').extract()

['Menu item 1', 'Menu item 2', 'Menu item 3', 'Menu item 4']

## Scraping a real website!

Now let's get to the best part: scraping a real website and crunching the results. We're going to use a website that's been set up to play with web scraping: [quotes.toscrape.com](http://quotes.toscrape.com)

But wait! Up to this point, you've been able to see the HTML code you're scraping in plain text. How do you figure out what xpaths or CSS selectors to use on a real website?

### Developer Tools

Most web browsers have tools for inspecting HTML in web pages built into them. You just need to figure out how to access those tools. Sometimes you have to turn them on first. They're not something an average user needs, so browsers may hide them to avoid confusion. Here are instructions for turning them on in some popular browsers:

1. [Chrome](https://developers.google.com/web/tools/chrome-devtools/)
2. [Firefox](https://developer.mozilla.org/en-US/docs/Tools/Settings)
3. [Safari](https://support.apple.com/en-ca/guide/safari/use-the-safari-develop-menu-sfri20948/mac)
4. [Internet Explorer](https://msdn.microsoft.com/en-us/library/dd565628(v=vs.85).aspx)

If you can't access the developer tools in your browser, you can still follow along in this notebook, as we've already captured the important paths. If you want to scrape other pages, though, these tools will help a lot.

Here's an example of how you access them via Google Chrome:

1. Open Chrome's Menu:
  <br/>
  <br/>
  ![chrome-menu](images/chrome-menu.png)
  <br/>
  <br/>
2. Navigate to "More Tools":
  <br/>
  <br/>
  ![chrome-menu](images/more-tools.png)
  <br/>
  <br/>

3. From there, choose "Developer Tools":
  <br/>
  <br/>
  ![chrome-menu](images/developer-tools.png)
  <br/>
  <br/>


You should see a screen like this (the developer tools may also show up on the right of your screen or in a separate window):

<br/>
<br/>
![chrome-menu](images/devtools-screen.png)
<br/>


From there, you can click the button in the upper left corner to enter a mode where any element you select on the web page highlights the corresponding HTML element in your developer tools:
<br/>

![inspect-element](images/inspect-element.png)

<br/>

Here's what it looks like when you hover over the quotation text on a page:

<br/>

![inspect-element](images/quote-span.png)

<br/>

And here's what your Developer Tools will show:

<br/>

![inspect-element](images/quote-location.png)

We want to get the author and the actual quote for every quotation on the page. Here are the paths you should find to those:

```
html body div.container div.row div.col-md-8 div.quote span.text
html body div.container div.row div.col-md-8 div.quote span small.author
```

Let's use those paths to tell our scraper where to get the data:

In [23]:
response = requests.get("http://quotes.toscrape.com/page/1/")
quotes = Selector(text=response.content).css(".quote")
for quote in quotes:
    text = quote.css(".text").xpath('.//text()').extract_first()
    author = quote.css(".author").xpath('.//text()').extract_first()
    print(text + " - " + author)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” - Albert Einstein
“It is our choices, Harry, that show what we truly are, far more than our abilities.” - J.K. Rowling
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” - Albert Einstein
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” - Jane Austen
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” - Marilyn Monroe
“Try not to become a man of success. Rather become a man of value.” - Albert Einstein
“It is better to be hated for what you are than to be loved for what you are not.” - André Gide
“I have not failed. I've just found 10,000 ways that won't work.” - Thomas A. Edison
“A woman is like a tea bag; you never know how strong it is until it's in hot water.” - Eleanor Roos

### Counting Quotes

Here's where things start to get interesting. Scraping a single page of quotes is probably not worth your time creating a script (unless you're learning!). If copying and pasting can quickly get you what you want, by all means, do that. But what if we wanted to grab more than a page of quotes so that we had lots of data to crunch? With that, we could find out the most popular authors on the site. Or if any particular words show up more often in quotes.

To do this, we're going to have to make a scraper that can navigate the website. As human beings, we would just keep clicking the "Next" button until we reached the end:

![nav-buttons](images/nav-buttons.png)

We can easily make our code do this too. If we can get our scraper to look at the HTML code that creates the "Next" button, we can find a link to the next page, and we can pass that, instead of `"http://quotes.toscrape.com/page/1/"` to our `requests.get` function.

Let's inspect the "Next" button by clicking the element selection button in our Developer Tools:

![nav-buttons](images/inspect-element.png)

And then clicking the "Next" button:

![nav-buttons](images/next-button.png)


If everything goes as planned, you should see something like this:


![nav-buttons](images/next-location.png)


Now we can see that the exact path to the unerlying link in the "Next" button is:

```
html body div.container div.row div.col-md-8 nav ul.pager li.next a
```

Specifically, we see the link to the next page within the `href` attribute of the `<a></a>` tag. So we know what we need to grab. How are we going to grab it? 

We could use the entire string above as a CSS selector. However, if we do that, we're counting on the navigation never changing location. But if we looked for every `<a></a>` tag, we'd get a bunch of links we didn't want. What parts of that heirarchy of elements really capture what the "Next" button is? Well, it's a navigation button, so it's not too unreasonable to expect that `nav` is likely to be a parent even if a lot of other things change. The `.next` class helps us tell between the "Next" and "Previous" buttons, so that's important too. We don't really care that it's on a list item (`<li></li>`) so we won't include that in the selector. And to get the `href` attribute, we need an `a` tag. By making sure that the `a` tag has a parent with `.next`, we can be fairly sure we're getting the right one. The creator of the page could certainly put multiple unrelated links under the `.next` parent, but the way the HTML is structured already suggests that they wouldn't be likely to do that.

Let's see if we can find the link to page 2 from page 1:

In [24]:
response = requests.get("http://quotes.toscrape.com/page/1/")
Selector(text=response.content).css("nav .next a").xpath("@href").extract_first()

'/page/2/'

Great! It looks like we'll have to prepend `http://quotes.toscrape.com` to our `href` value before we can actually use it, but that's not a big deal because it never changes. We can keep looking for the "Next" button, get the "href" value, and stop when we can't find a "Next" button.

#### Predicting Failure

It's good to play a little "What if?" before going on. What if we get the wrong link? Or what if the very last page contains a "Next" link that just puts us back at page 1? We don't want our scraper to go on forever. Let's pick a number of pages as an absolute maximum before we want our scraper to stop looking for more. We'll use 50 for now.

```python
stop_value = 50
```

#### Final scraping code

In [25]:
stop_value = 50

# To get rid of the beginning and ending quotation marks in the actual quotations.
def trim_quotation_marks(text):
    if text.startswith('“'):
        text = text[1:]
    if text.endswith('”'):
        text = text[:-1]
    return text

def scrape_pages(path, scraped=0):
    print("Scraping: " + path)

    response = requests.get("http://quotes.toscrape.com" + path)
    
    raw_quotes = Selector(text=response.content).css(".quote")
    quotes = []
    
    for quote in raw_quotes:
        text = quote.css(".text").xpath('.//text()').extract_first()
        # We already know this is a quotation. Let's get rid of those extra " marks
        text = trim_quotation_marks(text)
        author = quote.css(".author").xpath('.//text()').extract_first()
        quotes.append({'text': text, 'author': author})

    next_page = Selector(text=response.content).css("nav .next a").xpath("@href").extract_first()
    if next_page and scraped < stop_value:
        quotes = quotes + scrape_pages(next_page, scraped + 1)
        
    return quotes

all_quotes = scrape_pages("/page/1/")

# We'll show a preview
all_quotes[0:10]

Scraping: /page/1/
Scraping: /page/2/
Scraping: /page/3/
Scraping: /page/4/
Scraping: /page/5/
Scraping: /page/6/
Scraping: /page/7/
Scraping: /page/8/
Scraping: /page/9/
Scraping: /page/10/


[{'author': 'Albert Einstein',
  'text': 'The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.'},
 {'author': 'J.K. Rowling',
  'text': 'It is our choices, Harry, that show what we truly are, far more than our abilities.'},
 {'author': 'Albert Einstein',
  'text': 'There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.'},
 {'author': 'Jane Austen',
  'text': 'The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.'},
 {'author': 'Marilyn Monroe',
  'text': "Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring."},
 {'author': 'Albert Einstein',
  'text': 'Try not to become a man of success. Rather become a man of value.'},
 {'author': 'André Gide',
  'text': 'It is better to be hated for what you are than to be loved for what you are not.'},
 {'author'

#### Visualizing Author Contributions

Great! Now we have a real dataset! Let's visualize our top authors and words!

In [26]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import pandas as pd

authors = []
texts = []

for quote in all_quotes:
    authors.append(quote['author'])
    texts.append(quote['text'])

quotes = pd.DataFrame({'author':authors, 'text':texts})
author_counts = (quotes.groupby('author')
                       .count()
                       .reset_index()
                       .rename(columns={'text':'count'})
                       .sort_values(['count'], ascending=False))

init_notebook_mode(connected=True)

data = [go.Bar(x=author_counts['author'][0:10], y=author_counts['count'][0:10])]

iplot(data)

Looks like Albert Einstein and J.K. Rowling are favourites on this particular website.

How long do you think it would have taken to come up with this answer manually? Counting author contributions on a quotes page may seem trivial, but can you imagine other uses of this sort of analysis? Could it be used to show whether a website focuses more on particular subjects or people to get an idea of how biased it may or may not be?

#### Visualizing Words in Popular Quotations

Do certain words show up more than others in popular quotations? Let's find out!

Here's how you can go about it:

1. Mash all the text from all the quotations together.
2. Remove words we know are really common, like 'a', 'the', 'and', etc. We've created a function, called `interesting_words` to help you with this. It will return a list of words that aren't very common words.
3. Just as authors were counted, we can group by our words and count those, then sort them from highest to lowest.

Here's what we get:

In [27]:
all_quote_text = " ".join(quotes['text'])

wordset = pd.DataFrame(data={'word':interesting_words(all_quote_text),'count':1})

word_counts = (wordset.groupby('word')
                      .count()
                      .reset_index()
                      .sort_values(['count'], ascending=False))
word_counts[0:10]

,word,count
306,love,23
365,one,15
354,never,13
487,think,12
290,life,12
319,make,10
291,like,9
211,good,8
378,people,7
427,see,7


And now we can plot those words, just like we plotted the authors.

In [28]:
data = [go.Bar(x=reverse_column(word_counts['count'][0:10]), 
               y=reverse_column(word_counts['word'][0:10]),
               orientation = 'h')]

iplot(data)

Looks like "love" wins out, which isn't too surprising. It would be interesting to see the top words in a larger set of quotations, or if different types of quotations (humorous vs. inspiring, for example) had different top words. Feel free to try that on this or other data you can find. You have all the tools!

## Conclusion

You've just covered a lot of ground. You learned a little about HTML tags. You've learned how to grab specific items on a page. And you've learned how you can use information from `<a></a>` tags to navigate to other pages on a website. Finally, you've seen some of the questions you can answer through scraping that would be hard to answer otherwise. Congratulations!